In [1]:
# Mengimpor pustaka PyTorch untuk tugas deep learning dan pengolahan citra
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
# Memilih perangkat (device) yang akan digunakan, jika CUDA (GPU) tersedia, gunakan CUDA, jika tidak, gunakan CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Mendefinisikan kelas VGGNet sebagai turunan dari nn.Module
class VGGNet(nn.Module):
    def __init__(self):
        super(VGGNet, self).__init__()

        # Definisi layer-layer dari arsitektur VGGNet
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))

        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 10),  # 10 classes for MNIST
        )

    # Definisi langkah-langkah forward propagation
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [4]:
# Memuat dataset MNIST dan menentukan transformasi data
transform = transforms.Compose([
    transforms.Resize((28, 28)),  # Mengubah ukuran gambar menjadi (28, 28)
    transforms.ToTensor(),  # Mengubah gambar menjadi tensor PyTorch
])

# Membuat objek dataset pelatihan dan pengujian menggunakan dataset MNIST
train_dataset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# Membuat DataLoader untuk mempermudah pengelolaan batch data pada pelatihan dan pengujian
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 89489999.13it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 88874316.82it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 33548999.46it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 20180644.88it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
# Menginisialisasi model, fungsi loss, dan optimizer
model = VGGNet().to(device)  # Membuat instance dari kelas VGGNet dan memindahkannya ke perangkat yang telah ditentukan sebelumnya
criterion = nn.CrossEntropyLoss()  # Menggunakan fungsi loss CrossEntropyLoss untuk klasifikasi multikelas
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Menggunakan optimizer Adam dengan learning rate sebesar 0.001

In [6]:
# Pelatihan model
num_epochs = 5

for epoch in range(num_epochs):
    model.train()  # Mengaktifkan mode pelatihan pada model
    total_loss = 0.0

    # Iterasi untuk setiap batch pada data pelatihan
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Mengatur gradien parameter model menjadi 0

        outputs = model(inputs)  # Melakukan forward pass
        loss = criterion(outputs, labels)  # Menghitung nilai fungsi loss
        loss.backward()  # Melakukan backward pass
        optimizer.step()  # Optimasi parameter model

        total_loss += loss.item()

    # Menampilkan informasi hasil pelatihan pada setiap epoch
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

Epoch 1/5, Loss: 0.22661308444582665
Epoch 2/5, Loss: 0.06579108356891859
Epoch 3/5, Loss: 0.05726492838138797
Epoch 4/5, Loss: 0.044215179256350504
Epoch 5/5, Loss: 0.03938250152943146


In [7]:
# Pengujian model
model.eval()  # Mengubah model ke mode evaluasi (non-pelatihan)
total_correct = 0
total_samples = 0

with torch.no_grad():
    # Iterasi untuk setiap batch pada data uji
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)  # Melakukan forward pass
        _, predictions = torch.max(outputs, 1)  # Mengambil indeks kelas dengan nilai probabilitas tertinggi

        total_correct += (predictions == labels).sum().item()  # Menghitung jumlah prediksi yang benar
        total_samples += labels.size(0)  # Menghitung jumlah sampel total

# Menghitung dan menampilkan akurasi pada data uji
accuracy = total_correct / total_samples
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.9883
